# Predicting the condition of water wells in Tanzania

<img src="./images/water_fill.jpg" 
     align="left" 
     width="400" />

### Data Preparation

In [1]:
#Imports
import pandas as pd
import numpy as np

In [40]:
#Loading data
test_set_values = pd.read_csv('Data/test_set_values.csv')
training_set_labels = pd.read_csv('Data/training_set_labels.csv')
training_set_values = pd.read_csv('Data/training_set_values.csv')


#### Merging training set values and training set labels 

In [106]:
#inner merge
training_all = pd.merge(training_set_values, training_set_labels, on='id', how='inner')

In [107]:
training_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

### Data Cleaning - dealing with N/A values and removing unnecessary data columns

Looking at the merged training dataset above, the important columns for status of the wells might be x, x, x,
To make the data easier to look at, I chose to remove other columns. 

In [108]:
training_all.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [109]:
#Dropping the num_private column because there is no data on what this column means on the website source of the data
training_all.drop(columns='num_private', inplace=True)

In [110]:
# Replace specific values in the 'installer' column
training_all['installer'] = training_all['installer'].replace('Hesawa', 'HESAWA')
training_all['installer'] = training_all['installer'].replace('DANID', 'DANIDA')
training_all['installer'] = training_all['installer'].replace('Commu', "Community")

In [111]:
#changing the name of some the columns so they are more intuitive
training_all.rename(columns={'gps_height': 'well_altitude'}, inplace=True)
training_all.rename(columns={'wpt_name': 'water_point_name'}, inplace=True)
training_all.rename(columns={'payment': 'payment_type'}, inplace=True)
training_all.rename(columns={'payment_type': 'frequency_of_payment'}, inplace=True)
training_all.rename(columns={'funder': 'funding_source'}, inplace=True)


In [112]:
# Changing three options for functioning status of wells to two options - functional or needing repair
to_replace = ['functional needs repair']
new_value = 'functional'
training_all['status_group'] = training_all['status_group'].replace(to_replace, new_value)

In [113]:
training_all.head()

,id,amount_tsh,date_recorded,funding_source,well_altitude,installer,longitude,latitude,water_point_name,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [114]:
training_all.dropna(subset=['latitude'], inplace=True)
training_all.dropna(subset=['longitude'], inplace=True)
training_all = training_all.loc[training_all['longitude'] != 0]
training_all = training_all.loc[training_all['latitude'] != 0]

## Scheme Management - who operates the waterpoint

In [115]:
mgmt = training_all

In [116]:
mgmt['scheme_management'].value_counts()

VWC                 36143
WUG                  4249
Water authority      3151
WUA                  2882
Water Board          2747
Parastatal           1607
Private operator     1063
Company              1061
Other                 765
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [117]:
# drop n/a
mgmt.dropna(subset=['scheme_management'], inplace=True)

In [118]:
# drop "None" and "Other" values
mgmt = mgmt.loc[mgmt['scheme_management'] != 'None']
mgmt = mgmt.loc[mgmt['scheme_management'] != 'Other']

In [119]:
mgmt['status_group'].value_counts()

functional        32796
non functional    20276
Name: status_group, dtype: int64

In [120]:
# Encode scheme management column using one-hot encoding
mgmt_dummy = pd.get_dummies(mgmt, columns=['scheme_management'], prefix='scheme')
mgmt_dummy.head()

,id,amount_tsh,date_recorded,funding_source,well_altitude,installer,longitude,latitude,water_point_name,basin,...,scheme_Company,scheme_Parastatal,scheme_Private operator,scheme_SWC,scheme_Trust,scheme_VWC,scheme_WUA,scheme_WUG,scheme_Water Board,scheme_Water authority
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,Lake Nyasa,...,0,0,0,0,0,1,0,0,0,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,Pangani,...,0,0,0,0,0,1,0,0,0,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,...,0,0,0,0,0,1,0,0,0,0
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,Pangani,...,0,0,0,0,0,1,0,0,0,0
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,Internal,...,0,0,0,0,0,1,0,0,0,0


In [121]:
mgmt_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53072 entries, 0 to 59399
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       53072 non-null  int64  
 1   amount_tsh               53072 non-null  float64
 2   date_recorded            53072 non-null  object 
 3   funding_source           49500 non-null  object 
 4   well_altitude            53072 non-null  int64  
 5   installer                49502 non-null  object 
 6   longitude                53072 non-null  float64
 7   latitude                 53072 non-null  float64
 8   water_point_name         53072 non-null  object 
 9   basin                    53072 non-null  object 
 10  subvillage               52709 non-null  object 
 11  region                   53072 non-null  object 
 12  region_code              53072 non-null  int64  
 13  district_code            53072 non-null  int64  
 14  lga                   

In [122]:
mgmt_dummy.drop(columns='date_recorded', inplace=True)

In [123]:
# model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the dataset into features (X) and target (y)
X = mgmt_dummy[['scheme_Company', 'scheme_Parastatal', 'scheme_Private operator', 'scheme_SWC', 'scheme_Trust', 'scheme_WUA', 'scheme_WUG', 'scheme_Water Board', 'scheme_Water authority']]  # Drop the target column
y = mgmt_dummy['status_group']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.61


## Installer

In [124]:

# Replace 'installer' with the actual column name
installer_counts = training_all['installer'].value_counts()

# Get installer types with less than 2 instances
installers_to_remove = installer_counts[installer_counts < 500].index

# Filter the dataset to exclude rows with those installer types
installer_filtered = training_all[~training_all['installer'].isin(installers_to_remove)]


In [125]:
installer_filtered = installer_filtered.loc[installer_filtered['installer'] != '0']

In [126]:
installer_filtered['installer'].value_counts()

DWE                   14221
Government             1652
DANIDA                 1598
Community              1591
HESAWA                 1229
RWE                    1020
KKKT                    890
TCRS                    678
Central government      619
CES                     610
District Council        546
Name: installer, dtype: int64

In [127]:
installer_dummy = pd.get_dummies(installer_filtered, columns=['installer'], prefix='installer')
installer_dummy.head()

,id,amount_tsh,date_recorded,funding_source,well_altitude,longitude,latitude,water_point_name,basin,subvillage,...,installer_Central government,installer_Community,installer_DANIDA,installer_DWE,installer_District Council,installer_Government,installer_HESAWA,installer_KKKT,installer_RWE,installer_TCRS
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,39.172796,-4.765587,Tajiri,Pangani,Moa/Mwereme,...,0,0,0,1,0,0,0,0,0,0
11,50409,200.0,2013-02-18,Danida,1062,35.770258,-10.574175,Kwa Alid Nchimbi,Lake Nyasa,Ngondombwito,...,0,0,1,0,0,0,0,0,0,0
15,61848,0.0,2011-08-04,Rudep,1645,31.444121,-8.274962,Kwa Juvenal Ching'Ombe,Lake Tanganyika,Tunzi,...,0,0,0,1,0,0,0,0,0,0
16,48451,500.0,2011-07-04,Unicef,1703,34.642439,-9.106185,Kwa John Mtenzi,Rufiji,Kidudumo,...,0,0,0,1,0,0,0,0,0,0
17,58155,0.0,2011-09-04,Unicef,1656,34.569266,-9.085515,Kwa Rose Chaula,Rufiji,Yeriko,...,0,0,0,1,0,0,0,0,0,0


In [128]:
installer_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28236 entries, 5 to 59397
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            28236 non-null  int64  
 1   amount_tsh                    28236 non-null  float64
 2   date_recorded                 28236 non-null  object 
 3   funding_source                24693 non-null  object 
 4   well_altitude                 28236 non-null  int64  
 5   longitude                     28236 non-null  float64
 6   latitude                      28236 non-null  float64
 7   water_point_name              28236 non-null  object 
 8   basin                         28236 non-null  object 
 9   subvillage                    28036 non-null  object 
 10  region                        28236 non-null  object 
 11  region_code                   28236 non-null  int64  
 12  district_code                 28236 non-null  int64  
 13  l

In [130]:
# model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the dataset into features (X) and target (y)
X = installer_dummy[['installer_CES', 'installer_Central government', 'installer_Community', 'installer_DANIDA', 'installer_DWE', 'installer_District Council', 'installer_Government', 'installer_HESAWA', 'installer_KKKT', 'installer_RWE', 'installer_TCRS']]  # Drop the target column
y = installer_dummy['status_group']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.64


## Plotting waterpoints on a map

In [ ]:
import plotly.express as px
# Create a scatter_geo plot
fig = px.scatter_geo(training_all, lat='latitude', lon='longitude', color='quantity')

# Customize the layout
fig.update_geos(projection_type="mercator", showcoastlines=True)

# Show the plot
fig.show()

              status_group functional_status
0               functional        functional
1           non functional      needs repair
2  functional needs repair      needs repair


In [116]:
training_all.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,55523.000000,55523.000000,55523.000000,55523.000000,5.552300e+04,55523.000000,55523.000000,55523.000000,55523.000000,55523.000000
mean,37146.763557,333.537140,688.531239,34.087079,-5.721138e+00,0.493615,14.721845,5.650271,173.576014,1303.267781
std,21450.053863,3097.740715,698.667411,6.550291,2.923804e+00,12.446210,16.692759,9.576446,469.840330,950.764199
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18568.500000,0.000000,0.000000,33.165242,-8.564931e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37095.000000,0.000000,436.000000,34.912887,-5.035329e+00,0.000000,12.000000,3.000000,20.000000,1986.000000
75%,55696.500000,25.000000,1338.000000,37.210888,-3.331510e+00,0.000000,17.000000,5.000000,200.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.323402,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000
